# Optimize the model 

## Repeat Preprocessing from Starter_Code notebook

In [5]:
# Import our dependencies 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

# Call new DF to verify
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_count = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace =application_types_count[application_types_count<200].index.tolist()
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

class_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = class_counts[class_counts<1800].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummy_df = pd.get_dummies(application_df)

application_dummy_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [10]:
# Split our preprocessed data into our features and target arrays
X=application_dummy_df.drop(columns='IS_SUCCESSFUL')
y=application_dummy_df.IS_SUCCESSFUL

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train & Evaluate Optimized Model

The 3 optimization methods/attempts I chose are:

1. **Add another hidden layer** <br>
    I added the second hidden layer with 60 neurons, the `accuracy of model 1: 72.84%`, a loss of 0.06% from the original
    
2.  **Increase Neurons in Layers** <br>
    Increase the number of neurons in each layer to 150, 100, 50, 1
    `Accuracy of model 2: 72.86%`, slight improvement from model 1.

3. **Increase Neurons Significantly** <br>
    Seeing as method 2 of increasing neurons seemed to effectiveness, and after discussion with my tutor who explained that neuron limit really comes down to the limitations of your computer. Also that the provided starting neurons of 80, 30, 1 are in fact quite low.
    For method 3 I will use 300, 175, 100 , 1.
    `Accuracy of method 3: 72.84%`

4. **Reduce Epochs** <br>
    After analyzing the individual accuracy scores across all the epochs of method 3, I noticed a peak of 0.7412 at epochs 76 & 90. Therefore I will run method 3 again, this time with 80 epochs to hopefully limit overfitting. `Accuracy of method 4: 72.86%`, sligtht improvement but did not exceed the best of method 2.

5. **Add ANOTHER Hidden Layer**
    Adding a fifth layer, and dispersing the neurons to 300, 200, 100, 50 and 1(output). This I hope will decrease the overall complexity over a larger model "architecture". Also keeping the epochs at 80, and Neurons at 300, 200, 100, 50 & 1. `Accuracy of 5th model: 72.82%`
    
6. **RE-RUNNING Model #5** on model 2's neuron range of 150input - 1ouput.
    `Re-run model 5 accuracy: 72.82% ` Identical accuracy to method 5.


## METHOD 1

In [14]:
# METHOD #1
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_opt = tf.keras.models.Sequential()

# First hidden layer
nn_opt.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=43))

# Second hidden layer
nn_opt.add(tf.keras.layers.Dense(units=60, activation='relu'))

# Third hidden layer
nn_opt.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn_opt.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_opt.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 80)                3520      
                                                                 
 dense_9 (Dense)             (None, 60)                4860      
                                                                 
 dense_10 (Dense)            (None, 30)                1830      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 10241 (40.00 KB)
Trainable params: 10241 (40.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn_opt.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_nn_opt = nn_opt.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5680 - accuracy: 0.7202
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7296
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5522 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7308
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7325
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7338
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7337
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7347
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7356
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accura

In [16]:
# Evaluate MODEL #1 using the test data
model_loss, model_accuracy = nn_opt.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5642 - accuracy: 0.7284 - 594ms/epoch - 2ms/step
Loss: 0.5642269253730774, Accuracy: 0.728396475315094


## METHOD 2

In [17]:
# METHOD #2 
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_opt_two = tf.keras.models.Sequential()

# First hidden layer
nn_opt_two.add(tf.keras.layers.Dense(units=150, activation='relu', input_dim=43))

# Second hidden layer
nn_opt_two.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Third hidden layer
nn_opt_two.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn_opt_two.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_opt_two.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 150)               6600      
                                                                 
 dense_13 (Dense)            (None, 100)               15100     
                                                                 
 dense_14 (Dense)            (None, 50)                5050      
                                                                 
 dense_15 (Dense)            (None, 1)                 51        
                                                                 
Total params: 26801 (104.69 KB)
Trainable params: 26801 (104.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn_opt_two.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
nn_opt_two.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5676 - accuracy: 0.7221
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7287
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5528 - accuracy: 0.7305
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5501 - accuracy: 0.7321
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7331
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5478 - accuracy: 0.7325
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5470 - accuracy: 0.7336
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7345
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accura

In [19]:
# Evaluate MODEL #2 using the test data
model_loss, model_accuracy = nn_opt_two.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5877 - accuracy: 0.7286 - 576ms/epoch - 2ms/step
Loss: 0.5876883268356323, Accuracy: 0.7286297082901001


## METHOD 3

In [20]:
# METHOD #3
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_opt_three = tf.keras.models.Sequential()

# First hidden layer
nn_opt_three.add(tf.keras.layers.Dense(units=300, activation='relu', input_dim=43))

# Second hidden layer
nn_opt_three.add(tf.keras.layers.Dense(units=175, activation='relu'))

# Third hidden layer
nn_opt_three.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
nn_opt_three.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_opt_three.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 300)               13200     
                                                                 
 dense_17 (Dense)            (None, 175)               52675     
                                                                 
 dense_18 (Dense)            (None, 100)               17600     
                                                                 
 dense_19 (Dense)            (None, 1)                 101       
                                                                 
Total params: 83576 (326.47 KB)
Trainable params: 83576 (326.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Compile the model
nn_opt_three.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
nn_opt_three.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5675 - accuracy: 0.7234
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5568 - accuracy: 0.7288
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5526 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5510 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5500 - accuracy: 0.7323
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5491 - accuracy: 0.7331
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7329
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5474 - accuracy: 0.7344
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5466 - accuracy: 0.7345
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5462 - accura

In [22]:
# Evaluate MODEL #3 using the test data
model_loss, model_accuracy = nn_opt_three.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5824 - accuracy: 0.7284 - 723ms/epoch - 3ms/step
Loss: 0.5823505520820618, Accuracy: 0.728396475315094


## METHOD 4

In [23]:
# METHOD #4
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_opt_four = tf.keras.models.Sequential()

# First hidden layer
nn_opt_four.add(tf.keras.layers.Dense(units=300, activation='relu', input_dim=43))

# Second hidden layer
nn_opt_four.add(tf.keras.layers.Dense(units=175, activation='relu'))

# Third hidden layer
nn_opt_four.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
nn_opt_four.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_opt_four.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 300)               13200     
                                                                 
 dense_21 (Dense)            (None, 175)               52675     
                                                                 
 dense_22 (Dense)            (None, 100)               17600     
                                                                 
 dense_23 (Dense)            (None, 1)                 101       
                                                                 
Total params: 83576 (326.47 KB)
Trainable params: 83576 (326.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
# Compile the model
nn_opt_four.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
nn_opt_four.fit(X_train_scaled, y_train, epochs=80)

Epoch 1/80
804/804 [==============================] - 4s 3ms/step - loss: 0.5672 - accuracy: 0.7211
Epoch 2/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5562 - accuracy: 0.7301
Epoch 3/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5526 - accuracy: 0.7307
Epoch 4/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5511 - accuracy: 0.7307
Epoch 5/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5498 - accuracy: 0.7329
Epoch 6/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5486 - accuracy: 0.7341
Epoch 7/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5485 - accuracy: 0.7342
Epoch 8/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7338
Epoch 9/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5467 - accuracy: 0.7347
Epoch 10/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7357

In [25]:
# Evaluate MODEL #4 using the test data
model_loss, model_accuracy = nn_opt_four.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5744 - accuracy: 0.7286 - 618ms/epoch - 2ms/step
Loss: 0.5744339823722839, Accuracy: 0.7286297082901001


## METHOD 5

In [26]:
# METHOD #5
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_opt_five = tf.keras.models.Sequential()

# First hidden layer
nn_opt_five.add(tf.keras.layers.Dense(units=300, activation='relu', input_dim=43))

# Second hidden layer
nn_opt_five.add(tf.keras.layers.Dense(units=200, activation='relu'))

# Third hidden layer
nn_opt_five.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Fourth hidden layer
nn_opt_five.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn_opt_five.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_opt_five.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 300)               13200     
                                                                 
 dense_25 (Dense)            (None, 200)               60200     
                                                                 
 dense_26 (Dense)            (None, 100)               20100     
                                                                 
 dense_27 (Dense)            (None, 50)                5050      
                                                                 
 dense_28 (Dense)            (None, 1)                 51        
                                                                 
Total params: 98601 (385.16 KB)
Trainable params: 98601 (385.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model
nn_opt_five.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
nn_opt_five.fit(X_train_scaled, y_train, epochs=80)

Epoch 1/80
804/804 [==============================] - 5s 3ms/step - loss: 0.5679 - accuracy: 0.7215
Epoch 2/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5557 - accuracy: 0.7265
Epoch 3/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5527 - accuracy: 0.7315
Epoch 4/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5517 - accuracy: 0.7317
Epoch 5/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5502 - accuracy: 0.7331
Epoch 6/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5486 - accuracy: 0.7334
Epoch 7/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5479 - accuracy: 0.7324
Epoch 8/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5473 - accuracy: 0.7336
Epoch 9/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5469 - accuracy: 0.7342
Epoch 10/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5464 - accuracy: 0.7343

In [28]:
# Evaluate MODEL #5 using the test data
model_loss, model_accuracy = nn_opt_five.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5837 - accuracy: 0.7282 - 682ms/epoch - 3ms/step
Loss: 0.5837388038635254, Accuracy: 0.7281632423400879


## METHOD 6

In [29]:
# METHOD #6
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_opt_six = tf.keras.models.Sequential()

# First hidden layer
nn_opt_six.add(tf.keras.layers.Dense(units=150, activation='relu', input_dim=43))

# Second hidden layer
nn_opt_six.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Third hidden layer
nn_opt_six.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Fourth hidden layer
nn_opt_six.add(tf.keras.layers.Dense(units=25, activation='relu'))

# Output layer
nn_opt_six.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_opt_six.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 150)               6600      
                                                                 
 dense_30 (Dense)            (None, 100)               15100     
                                                                 
 dense_31 (Dense)            (None, 50)                5050      
                                                                 
 dense_32 (Dense)            (None, 25)                1275      
                                                                 
 dense_33 (Dense)            (None, 1)                 26        
                                                                 
Total params: 28051 (109.57 KB)
Trainable params: 28051 (109.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# Compile the model
nn_opt_six.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
nn_opt_six.fit(X_train_scaled, y_train, epochs=80)

Epoch 1/80
804/804 [==============================] - 4s 3ms/step - loss: 0.5672 - accuracy: 0.7221
Epoch 2/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5549 - accuracy: 0.7293
Epoch 3/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7305
Epoch 4/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7314
Epoch 5/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7328
Epoch 6/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7325
Epoch 7/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7338
Epoch 8/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7341
Epoch 9/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7356
Epoch 10/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7353

In [32]:
# Evaluate MODEL #6 using the test data
model_loss, model_accuracy = nn_opt_six.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5673 - accuracy: 0.7282 - 470ms/epoch - 2ms/step
Loss: 0.5673278570175171, Accuracy: 0.7281632423400879
